# Test using Globus for transferring files from Voltaiq to external sources

https://globus-sdk-python.readthedocs.io/en/stable/examples/minimal_transfer_script.html

In [13]:
import globus_sdk
from globus_sdk.scopes import TransferScopes
from globus_sdk import TransferClient, LocalGlobusConnectPersonal


In [7]:
CLIENT_ID = '61338d24-54d5-408f-a10d-66c06b59f6d2'

auth_client = globus_sdk.NativeAppAuthClient(CLIENT_ID)

# requested_scopes specifies a list of scopes to request
# instead of the defaults, only request access to the Transfer API
auth_client.oauth2_start_flow(requested_scopes=TransferScopes.all)
authorize_url = auth_client.oauth2_get_authorize_url()
print(f"Please go to this URL and login:\n\n{authorize_url}\n")

Please go to this URL and login:

https://auth.globus.org/v2/oauth2/authorize?client_id=61338d24-54d5-408f-a10d-66c06b59f6d2&redirect_uri=https%3A%2F%2Fauth.globus.org%2Fv2%2Fweb%2Fauth-code&scope=urn%3Aglobus%3Aauth%3Ascope%3Atransfer.api.globus.org%3Aall&state=_default&response_type=code&code_challenge=qZlFdExAyXKYXt_wQ4wWFxOQTRJjTQCyK77xOgbu2g8&code_challenge_method=S256&access_type=online



In [8]:
auth_code = '5lbUK2Cvu7BzNrFVcPfD8tLJxORDsK'

In [9]:
tokens = auth_client.oauth2_exchange_code_for_tokens(auth_code)
transfer_tokens = tokens.by_resource_server["transfer.api.globus.org"]

In [10]:
# construct an AccessTokenAuthorizer and use it to construct the
# TransferClient
transfer_client = globus_sdk.TransferClient(
    authorizer=globus_sdk.AccessTokenAuthorizer(transfer_tokens["access_token"])
)

In [11]:
# Globus Tutorial Endpoint 1
source_endpoint_id = "ddb59aef-6d04-11e5-ba46-22000b92c6ec"
# Globus Tutorial Endpoint 2
dest_endpoint_id = "ddb59af0-6d04-11e5-ba46-22000b92c6ec"

In [12]:
# create a Transfer task consisting of one or more items
task_data = globus_sdk.TransferData(
    transfer_client, source_endpoint_id, dest_endpoint_id
)
task_data.add_item(
    "/share/godata/file1.txt",  # source
    "/~/minimal-example-transfer-script-destination.txt",  # dest
)

# submit, getting back the task ID
task_doc = transfer_client.submit_transfer(task_data)
task_id = task_doc["task_id"]
print(f"submitted transfer, task_id={task_id}")

submitted transfer, task_id=8ad97130-a240-11ed-a2a4-8383522b48d9


# Set up local endpoint

In [16]:
local_ep = LocalGlobusConnectPersonal()

ep_id = local_ep.endpoint_id

for file in transfer_client.operation_ls(ep_id):
    print("Local file: ", f['name'])

TransferAPIError: ('GET', 'https://transfer.api.globus.org/v0.10/operation/endpoint/None/ls', 'Bearer', 404, 'EndpointNotFound', "No such endpoint with legacy name 'None'", 'pmw3GvudD')

In [23]:
local_ep.endpoint_id